In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


In [3]:
import glob
import pandas as pd

enrol_files = glob.glob(
    r"C:\Users\Tanaya\Downloads\aadhaar-enrollment-main\aadhaar-enrollment-main\aadhaar-enrollment\data\enrollment\*.csv"
)

print(enrol_files)

enrol = pd.concat(
    [pd.read_csv(file) for file in enrol_files],
    ignore_index=True
)

demo_files = glob.glob(
    r"C:\Users\Tanaya\Downloads\aadhaar-enrollment-main\aadhaar-enrollment-main\aadhaar-enrollment\data\demographic\*.csv"
)

# Check if files are detected
print("Demographic files found:", demo_files)

# Read and combine all demographic CSVs
demo = pd.concat(
    [pd.read_csv(file) for file in demo_files],
    ignore_index=True
)

# Quick check
print("Demographic shape:", demo.shape)



['C:\\Users\\Tanaya\\Downloads\\aadhaar-enrollment-main\\aadhaar-enrollment-main\\aadhaar-enrollment\\data\\enrollment\\api_data_aadhar_enrolment_0_500000.csv', 'C:\\Users\\Tanaya\\Downloads\\aadhaar-enrollment-main\\aadhaar-enrollment-main\\aadhaar-enrollment\\data\\enrollment\\api_data_aadhar_enrolment_1000000_1006029.csv', 'C:\\Users\\Tanaya\\Downloads\\aadhaar-enrollment-main\\aadhaar-enrollment-main\\aadhaar-enrollment\\data\\enrollment\\api_data_aadhar_enrolment_500000_1000000.csv']
Demographic files found: ['C:\\Users\\Tanaya\\Downloads\\aadhaar-enrollment-main\\aadhaar-enrollment-main\\aadhaar-enrollment\\data\\demographic\\api_data_aadhar_demographic_0_500000.csv', 'C:\\Users\\Tanaya\\Downloads\\aadhaar-enrollment-main\\aadhaar-enrollment-main\\aadhaar-enrollment\\data\\demographic\\api_data_aadhar_demographic_1000000_1500000.csv', 'C:\\Users\\Tanaya\\Downloads\\aadhaar-enrollment-main\\aadhaar-enrollment-main\\aadhaar-enrollment\\data\\demographic\\api_data_aadhar_demographic

In [6]:
# Create total enrolment per row
enrol['total_enrolment'] = (
    enrol['age_0_5'] +
    enrol['age_5_17'] +
    enrol['age_18_greater']
)

enrol.head()


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,total_enrolment
0,02-03-2025,Meghalaya,East Khasi Hills,793121,11,61,37,109
1,09-03-2025,Karnataka,Bengaluru Urban,560043,14,33,39,86
2,09-03-2025,Uttar Pradesh,Kanpur Nagar,208001,29,82,12,123
3,09-03-2025,Uttar Pradesh,Aligarh,202133,62,29,15,106
4,09-03-2025,Karnataka,Bengaluru Urban,560016,14,16,21,51


In [7]:
enrol_agg = (
    enrol
    .groupby(['state', 'district'])
    .agg(
        total_enrolment=('total_enrolment', 'sum'),
        age_0_5=('age_0_5', 'sum'),
        age_5_17=('age_5_17', 'sum'),
        age_18_greater=('age_18_greater', 'sum')
    )
    .reset_index()
)

enrol_agg.head()


,state,district,total_enrolment,age_0_5,age_5_17,age_18_greater
0,100000,100000,218,0,1,217
1,Andaman & Nicobar Islands,Andamans,75,70,5,0
2,Andaman & Nicobar Islands,Nicobars,1,1,0,0
3,Andaman & Nicobar Islands,South Andaman,38,38,0,0
4,Andaman and Nicobar Islands,Nicobar,75,64,11,0


In [11]:
print(demo.columns)
demo_agg = (
    demo
    .groupby(['state', 'district'])
    .agg(
        demo_age_5_17=('demo_age_5_17', 'mean'),
        demo_age_17=('demo_age_17_', 'mean')
    )
    .reset_index()
)

demo_agg.head()


Index(['date', 'state', 'district', 'pincode', 'demo_age_5_17',
       'demo_age_17_'],
      dtype='object')


,state,district,demo_age_5_17,demo_age_17
0,100000,100000,0.000000,1.000000
1,Andaman & Nicobar Islands,Andamans,0.022293,2.366242
2,Andaman & Nicobar Islands,Nicobars,0.000000,1.000000
3,Andaman & Nicobar Islands,South Andaman,0.030769,1.533333
4,Andaman and Nicobar Islands,Nicobar,0.453125,5.695312


In [12]:
data = enrol_agg.merge(
    demo_agg,
    on=['state', 'district'],
    how='left'
)

data.shape


(1070, 8)

In [15]:
data['total_population'] = data['age_0_5'] + data['age_5_17'] + data['age_18_greater']

data['absolute_gap'] = data['total_population'] - data['total_enrolment']

data['gap_percentage'] = (
    data['absolute_gap'] / data['total_population']
) * 100


In [16]:
print(data.columns)

Index(['state', 'district', 'total_enrolment', 'age_0_5', 'age_5_17',
       'age_18_greater', 'demo_age_5_17', 'demo_age_17', 'total_population',
       'absolute_gap', 'gap_percentage'],
      dtype='object')


In [17]:
data = data.dropna()


In [18]:
#Predict future Aadhaar enrolment growth at district level
# Identify districts at risk of low enrolment growth
X = data[['age_0_5', 'age_5_17', 'age_18_greater']]
y = data['gap_percentage']


In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [20]:
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    random_state=42
)

rf.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples 

In [21]:
y_pred = rf.predict(X_test)


In [22]:
print("MAE :", mean_absolute_error(y_test, y_pred))
print("R2  :", r2_score(y_test, y_pred))


MAE : 0.0
R2  : 1.0


In [23]:
importance = pd.Series(
    rf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

importance


age_0_5           0.0
age_5_17          0.0
age_18_greater    0.0
dtype: float64

In [24]:
data['predicted_gap'] = rf.predict(X)


C:\Users\Tanaya\AppData\Local\Temp\ipykernel_78932\2546689520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['predicted_gap'] = rf.predict(X)


In [25]:
high_gap_districts = (
    data.groupby(['state', 'district'])['predicted_gap']
    .mean()
    .reset_index()
    .sort_values('predicted_gap', ascending=False)
)

high_gap_districts.head(10)


,state,district,predicted_gap
0,100000,100000,0.0
1,Andaman & Nicobar Islands,Andamans,0.0
2,Andaman & Nicobar Islands,Nicobars,0.0
3,Andaman & Nicobar Islands,South Andaman,0.0
4,Andaman and Nicobar Islands,Nicobar,0.0
5,Andaman and Nicobar Islands,North And Middle Andaman,0.0
6,Andaman and Nicobar Islands,South Andaman,0.0
7,Andhra Pradesh,Adilabad,0.0
8,Andhra Pradesh,Alluri Sitharama Raju,0.0
9,Andhra Pradesh,Anakapalli,0.0
